In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os

In [6]:
# Очистите список перед началом сбора данных
items_urls = []

for i in tqdm(range(1, 316)):
    if i == 1:
        end = '/'
    else:
        end = f'&page={i}'
    link = f'https://kino.mail.ru/series/all/?order=rate_count&year=1916&year=2024{end}'
    content = requests.get(link).content
    soup = BeautifulSoup(content, 'lxml')
    series = soup.find_all(class_='link link_inline link-holder link-holder_itemevent link-holder_itemevent_small')

    for item in series:
        item_page_url = item.get('href')
        # Проверьте, не был ли URL уже добавлен
        full_url = 'https://kino.mail.ru' + item_page_url
        if full_url not in items_urls:
            items_urls.append(full_url)

# Создайте DataFrame и сохраните в .csv
urls = pd.DataFrame({'url': items_urls})
urls.to_csv('urls_mail_ru.csv', index=False)

print(f'Количество уникальных URL-адресов: {len(items_urls)}')

100%|█████████████████████████████████████████| 315/315 [04:25<00:00,  1.19it/s]

Количество уникальных URL-адресов: 15750


In [3]:
df = pd.read_csv("/home/artemiy/recsys_project/data/urls_mail_ru.csv")
items_urls = df['url'].tolist()

# Разделите URL-адреса на переменные
l1, l2 = items_urls[:7500], items_urls[7500:15000]
l3 = items_urls[15000:]

In [4]:
items_urls[0]

'https://kino.mail.ru/series_756333_velikolepnii_vek/'

In [32]:
# Убедимся, что директория для сохранения существует
output_dir = "/home/artemiy/recsys_project/data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Ваши списки и переменные
titles = []
sources = []
descriptions = []
ganres = []
years = []
posters = []
casts = []
ratings = []
s = 1

# Ваш цикл для парсинга страниц
for url in tqdm(l1):
    try:
        text = requests.get(url).text
        soup = BeautifulSoup(text, "lxml")
        try:
            title = soup.find("h1", class_="text text_bold_giant color_white").text
            titles.append(title[: title.find(" (сери")])
        except Exception as e:
            print("Ошибочка в title:", e)
            titles.append(None)

        # Тянем постер
        try:
            picture_url = soup.find("meta", itemprop="image")["content"]
            posters.append(picture_url)
        except Exception as e:
            print("Ошибочка в poster:", e)
            posters.append(None)

        # Тянем жанры
        try:
            ganre = soup.find_all("span", class_="badge__text")
            helper = []
            for i in ganre:
                helper.append(i.text)
            ganres.append(helper)
        except Exception as e:
            print("Ошибочка в ganres:", e)
            ganres.append(None)

        # Тянем cast
        try:
            cast = soup.find_all(
                "span", class_="p-truncate__inner js-toggle__truncate-inner"
            )
            a = []
            for act in cast:
                a.append(act.text)
            casts.append(a[1:4])
        except Exception as e:
            print("Ошибочка в cast:", e)
            casts.append(None)

        try:
            description = soup.find(
                "div",
                class_="text text_inline text_light_medium text_fixed valign_baseline p-movie-info__description-text",
            ).text
            descriptions.append(description.replace("\xa0", " "))
        except Exception as e:
            print("Ошибочка в description:", e)
            descriptions.append(None)

        # Год
        try:
            year = soup.find("span", class_="nowrap").text
            years.append(year.split()[:3])
        except Exception as e:
            print("Ошибочка в year:", e)
            years.append(None)

        # IMDB
        try:
            rating = float(
                soup.find("span", class_="text text_bold_huge text_fixed").text
            )
            ratings.append(rating)
        except Exception as e:
            print("Ошибочка в rating:", e)
            ratings.append(None)

        sources.append(url)

        # Сохраняем данные по 500 записей или в конце списка
        if (len(sources) % 500 == 0) or (l1[-1] == url):
            data = pd.DataFrame(
                {
                    "url": sources,
                    "poster": posters,
                    "title": titles,
                    "ganres": ganres,
                    "description": descriptions,
                    "year": years,
                    "rating": ratings,
                    "cast": casts,
                }
            )
            print(f"{len(data)} saved")

            name = f"{output_dir}/data_{s}.csv"
            data.to_csv(name, index=False)
            print(f"Saved to {name}")

            s += 1
            titles.clear()
            sources.clear()
            descriptions.clear()
            ganres.clear()
            years.clear()
            posters.clear()
            casts.clear()
            ratings.clear()

    except requests.exceptions.RequestException as e:
        print(f"Link number: {l1.index(url)}, link: {url}, error: {e}")
# Сохраняем оставшиеся данные в последний файл
if titles or sources or descriptions or ganres or years or posters or casts or ratings:
    data = pd.DataFrame(
        {
            "url": sources,
            "poster": posters,
            "title": titles,
            "ganres": ganres,
            "description": descriptions,
            "year": years,
            "rating": ratings,
            "cast": casts,
        }
    )
    name = f"{output_dir}/data_{s}.csv"
    data.to_csv(name, index=False)
    print(f"Remaining {len(data)} records saved to {name}")

  7%|▋         | 500/7500 [07:18<1:36:53,  1.20it/s]

500 saved
Saved to /home/artemiy/recsys_project/data/data_1.csv


 13%|█▎        | 1000/7500 [13:59<1:35:51,  1.13it/s]

500 saved
Saved to /home/artemiy/recsys_project/data/data_2.csv


 20%|██        | 1500/7500 [20:35<1:16:57,  1.30it/s]

...

Ошибочка в rating: 'NoneType' object has no attribute 'text'


100%|█████████▉| 7498/7500 [1:37:54<00:01,  1.56it/s]

Ошибочка в rating: 'NoneType' object has no attribute 'text'


100%|██████████| 7500/7500 [1:37:56<00:00,  1.28it/s]

Ошибочка в rating: 'NoneType' object has no attribute 'text'
500 saved
Saved to /home/artemiy/recsys_project/data/data_15.csv


In [33]:
# Директория с файлами
input_dir = "/home/artemiy/recsys_project/data"

# Список всех файлов в директории
files = [f for f in os.listdir(input_dir) if f.endswith(".csv")]

# Создаем пустой DataFrame
combined_data = pd.DataFrame()

# Читаем и объединяем все файлы
for file in files:
    file_path = os.path.join(input_dir, file)
    df = pd.read_csv(file_path)
    combined_data = pd.concat([combined_data, df], ignore_index=True)

# Сохраняем объединенные данные в единый файл
output_file = os.path.join(input_dir, "combined_data.csv")
combined_data.to_csv(output_file, index=False)
print(f"Объединенные данные сохранены в {output_file}")

Объединенные данные сохранены в /home/artemiy/recsys_project/data/combined_data.csv


In [34]:
# Ваши списки и переменные
urls = combined_data["url"]
ratings = []

# Парсинг рейтингов
for url in tqdm(urls):
    try:
        text = requests.get(url).text
        soup = BeautifulSoup(text, "lxml")
        try:
            rating = float(
                soup.find("span", class_="text text_bold_huge text_fixed").text
            )
            ratings.append(rating)
        except:
            ratings.append(None)
    except requests.exceptions.RequestException as e:
        print(f"Link: {url}, error: {e}")
        ratings.append(None)

# Добавляем новые рейтинги в DataFrame
combined_data["rating"] = ratings

# Сохраняем обновленные данные в файл
output_file_with_ratings = os.path.join(input_dir, "combined_data_with_ratings.csv")
combined_data.to_csv(output_file_with_ratings, index=False)
print(f"Обновленные данные с рейтингами сохранены в {output_file_with_ratings}")

100%|██████████| 23250/23250 [4:57:18<00:00,  1.30it/s]   

Обновленные данные с рейтингами сохранены в /home/artemiy/recsys_project/data/combined_data_with_ratings.csv


In [41]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15750 entries, 0 to 20749
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          15750 non-null  object 
 1   poster       5000 non-null   object 
 2   title        5000 non-null   object 
 3   ganres       5000 non-null   object 
 4   description  5000 non-null   object 
 5   year         5000 non-null   object 
 6   rating       5741 non-null   float64
 7   cast         5000 non-null   object 
dtypes: float64(1), object(7)
memory usage: 1.1+ MB


In [40]:
combined_data['title'].duplicated().sum()

np.int64(10944)

In [38]:
combined_data = combined_data.drop_duplicates(subset=['url'])

In [39]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15750 entries, 0 to 20749
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          15750 non-null  object 
 1   poster       5000 non-null   object 
 2   title        5000 non-null   object 
 3   ganres       5000 non-null   object 
 4   description  5000 non-null   object 
 5   year         5000 non-null   object 
 6   rating       5741 non-null   float64
 7   cast         5000 non-null   object 
dtypes: float64(1), object(7)
memory usage: 1.1+ MB
